<span style="font-size: 250%;color:white;background:green"> przykład ze internetu #2


https://benalexkeen.com/linear-programming-with-python-and-pulp-part-5/
https://www.kaggle.com/parvezrh/production-and-supply

<span style="font-size: 250%;color:green;background:grey">ZASTOSOWANIE DATAFRAME
    
    W tym przykładzie rozwiążemy problem z harmonogramem. Posiadamy 2 offshore zakłady produkcyjne w 2 lokalizacjach i szacujemy zapotrzebowanie na nasze produkty.

Chcemy opracować harmonogram produkcji z obu zakładów, który spełnia nasze wymagania przy najniższych kosztach.

Fabryka może znajdować się w 2 stanach:

    Wył. - produkuje zero jednostek
    On - Produkcja pomiędzy minimalną i maksymalną zdolnością produkcyjną


    Obie fabryki mają koszty stałe, które są ponoszone tak długo, jak fabryka działa, oraz koszty zmienne, czyli koszt na jednostkę produkcji. Te zmieniają się z miesiąca na miesiąc.

Wiemy również, że fabryka B jest wyłączona z powodu konserwacji w miesiącu 5.

Zaczniemy od zaimportowania naszych danych.

In [1]:
import pandas as pd
import pulp

In [3]:
factories = pd.read_csv('/home/wojciech/Pulpit/1/datasets_27428_34967_Cost_Analysis.csv', index_col=['Month', 'Factory'])
factories

Max_Capacity  Min_Capacity  Variable_Costs  Fixed_costs
Month Factory                                                         
1     A              100000         20000              10          500
      B               50000         20000               5          600
2     A              110000         20000              11          500
      B               55000         20000               4          600
3     A              120000         20000              12          500
      B               60000         20000               3          600
4     A              145000         20000               9          500
      B              100000         20000               5          600
5     A              160000         20000               8          500
      B                   0             0               0            0
6     A              140000         20000               8          500
      B               70000         20000               6          600
7     A              155000         20000               5          500
      B               60000         20000               4          600
8     A              200000         20000               7          500
      B              100000         20000               6          600
9     A              210000         20000               9          500
      B              100000         20000               8          600
10    A              197000         20000              10          500
      B              100000         20000              11          600
11    A               80000         20000               8          500
      B               12000         20000              10          600
12    A              150000         20000               8          500
      B              150000         20000              12          600

Zaimportujemy również nasze dane dotyczące zapotrzebowania

In [4]:
demand = pd.read_csv('/home/wojciech/Pulpit/1/datasets_27428_34967_Demand.csv', index_col=['Month'])
demand

,Demand
Month,
1,120000
2,100000
3,130000
4,130000
5,140000
6,130000
7,150000
8,170000
9,200000


Ponieważ mamy koszty stałe i koszty zmienne, będziemy musieli modelować zarówno produkcję, jak i stan fabryki, tj. Czy jest włączona czy wyłączona.

Produkcja jest modelowana jako zmienna całkowita.

Mamy wartość produkcji dla każdego miesiąca dla każdej fabryki, która jest określona przez krotki naszego indeksu DataFrame pand z wieloma indeksami.

In [5]:
production = pulp.LpVariable.dicts("production",
                                     ((month, factory) for month, factory in factories.index),
                                     lowBound=0,
                                     cat='Integer')

In [6]:
production

{(1, 'A'): production_(1,_'A'),
 (1, 'B'): production_(1,_'B'),
 (2, 'A'): production_(2,_'A'),
 (2, 'B'): production_(2,_'B'),
 (3, 'A'): production_(3,_'A'),
 (3, 'B'): production_(3,_'B'),
 (4, 'A'): production_(4,_'A'),
 (4, 'B'): production_(4,_'B'),
 (5, 'A'): production_(5,_'A'),
 (5, 'B'): production_(5,_'B'),
 (6, 'A'): production_(6,_'A'),
 (6, 'B'): production_(6,_'B'),
 (7, 'A'): production_(7,_'A'),
 (7, 'B'): production_(7,_'B'),
 (8, 'A'): production_(8,_'A'),
 (8, 'B'): production_(8,_'B'),
 (9, 'A'): production_(9,_'A'),
 (9, 'B'): production_(9,_'B'),
 (10, 'A'): production_(10,_'A'),
 (10, 'B'): production_(10,_'B'),
 (11, 'A'): production_(11,_'A'),
 (11, 'B'): production_(11,_'B'),
 (12, 'A'): production_(12,_'A'),
 (12, 'B'): production_(12,_'B')}

Status fabryki jest modelowany jako zmienna binarna. Będzie mieć wartość 1, jeśli fabryka jest włączona i wartość 0, gdy fabryka jest wyłączona.

Zmienne binarne są takie same jak zmienne całkowite, ale ograniczone do wartości> = 0 i <= 1

Ponownie ma to wartość dla każdego miesiąca dla każdej fabryki, ponownie podaną przez indeks naszej ramki DataFrame

In [7]:
factory_status = pulp.LpVariable.dicts("factory_status",
                                     ((month, factory) for month, factory in factories.index),
                                     cat='Binary')

Tworzymy instancję naszego modelu i używamy LpMinimize, ponieważ celem jest minimalizacja kosztów.

In [9]:
model = pulp.LpProblem("Cost_minimising_scheduling_problem", pulp.LpMinimize)

In [10]:
print(model)

Cost_minimising_scheduling_problem:
MINIMIZE
None
VARIABLES



W naszej funkcji celu uwzględniamy nasze 2 koszty:

    Nasze koszty zmienne to iloczyn kosztów zmiennych na jednostkę i produkcji
    Nasze koszty stałe to stan fabryczny - 1 (włączony) lub 0 (wyłączony) - pomnożony przez stały koszt produkcji

In [11]:
model += pulp.lpSum(
    [production[month, factory] * factories.loc[(month, factory), 'Variable_Costs'] for month, factory in factories.index]
    + [factory_status[month, factory] * factories.loc[(month, factory), 'Fixed_costs'] for month, factory in factories.index]
)

## Budujemy nasze ograniczenia

In [13]:
months = demand.index
for n in months:
    model += production[(n, 'A')] + production[(n, 'B')] == demand.loc[n, 'Demand']

Problem, z którym się tutaj spotykamy, polega na tym, że w programowaniu liniowym nie możemy używać ograniczeń warunkowych.

Na przykład nie możemy dodać do naszego modelu, że jeśli fabryka jest poza fabryką, musi mieć status 0, a jeśli jest w fabryce, musi wynosić 1. Jednak zanim rozwiążemy nasz model, nie wiemy, czy fabryka będzie włączony lub wyłączony w danym miesiącu.

W tym przypadku skonstruuj ograniczenia, które mają minimalne i maksymalne pojemności, które są stałymi zmiennymi, które mnożymy przez stan fabryki.

Teraz albo stan fabryczny to 0 i:

min_production ≥ 0
max_production ≤ 0
Lub stan fabryczny to 1 i:

min_production ≤ min_capacity
max_production ≤ max_capacity
(W niektórych przypadkach możemy użyć więzów liniowych do modelowania instrukcji warunkowych, omówimy to w części 6)

In [14]:
for month, factory in factories.index:
    min_production = factories.loc[(month, factory), 'Min_Capacity']
    max_production = factories.loc[(month, factory), 'Max_Capacity']
    model += production[(month, factory)] >= min_production * factory_status[month, factory]
    model += production[(month, factory)] <= max_production * factory_status[month, factory]

In [15]:
model += factory_status[5, 'B'] == 0
model += production[5, 'B'] == 0

In [16]:
model.solve()
pulp.LpStatus[model.status]

'Infeasible'

Przyjrzyjmy się optymalnemu harmonogramowi produkcji na każdy miesiąc z każdej fabryki. Aby ułatwić przeglądanie, wyślemy dane do pandy DataFrame.

In [17]:
output = []
for month, factory in production:
    var_output = {
        'Month': month,
        'Factory': factory,
        'Production': production[(month, factory)].varValue
        #'Factory Status': factory_status[(month, factory)].varValue
    }
    output.append(var_output)
output_df = pd.DataFrame.from_records(output).sort_values(['Month', 'Factory'])
output_df.set_index(['Month', 'Factory'], inplace=True)
#output_df.set_index(['Month', 'Factory'])
output_df

Production
Month Factory            
1     A           70000.0
      B           50000.0
2     A           45000.0
      B           55000.0
3     A           70000.0
      B           60000.0
4     A           30000.0
      B          100000.0
5     A          140000.0
      B               0.0
6     A           60000.0
      B           70000.0
7     A           90000.0
      B           60000.0
8     A           70000.0
      B          100000.0
9     A          100000.0
      B          100000.0
10    A          190000.0
      B               0.0
11    A           80000.0
      B           60000.0
12    A          100000.0
      B               0.0

In [18]:
print (pulp.value(model.objective))

12906420.43077
